In [142]:
# Main Code for updating Entity and Key Phrases

import os
import math
import json
import sys
import uuid
import pandas as pd
from azure.cosmos import exceptions, CosmosClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
endpoint = os.environ.get('<text-analytics-endpoint>')
key = os.environ.get('<key-for-text-analytics-endpoint>')

text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))


uri = '<endpoint-for-cosmosdb>'
key = '<key-for-cosmosdb>'
client = CosmosClient(uri, key)

database = client.get_database_client('kym')


container2 = database.get_container_client('text_analytics')
 
###### For fetching all reactions from kymchat-messages ######
reactions = list(container.query_items(
    query="SELECT  m.reaction.response, m.requestMessageId, m.timestamp FROM c join m in c.messages where m.sender = 'kym' and is_defined(m.reaction) and m.reaction.response != ''",
   
    enable_cross_partition_query=True
))

reactionlist = []


###### To perform sentiment analysis on the fetched responses

for reaction in reactions:
    
    sentiment = text_analytics_client.analyze_sentiment([reaction['response']])
    reactiondict = {
            'requestMessageId': reaction['requestMessageId'],
            'timestamp':reaction['timestamp'],
            'sentiment_output':sentiment[0]['sentiment']
    }
    reactionlist.append(reactiondict)
    

####### Script for adding the Sentiment_Outcome to the existing dictionaries in text analytics######
for reaction in reactionlist:
    query = "select * from c where c.messageid=@requestMessageId"
    documents = list(container2.query_items(
        query=query,
        parameters=[

            {"name":"@requestMessageId", "value": reaction['requestMessageId']},

        ],

        enable_cross_partition_query=True

    ))
    
    for document in documents:
        #print(document)
        document['Sentiment_Outcome'] = reaction['sentiment_output']
        container2.replace_item(item=document['id'], body=document)